# Plot entire city neighborhood by neighborhood

_Tamas Budavari_ <br>
<budavari@jhu.edu> <br>
2018-08-30


In [ ]:
#import warnings
#warnings.filterwarnings('ignore')

#%pylab inline
from SciServer import CasJobs
import pandas
#from shapely.geometry import Polygon
from shapely import wkt
from matplotlib import pyplot as plt
import numpy as np
import time

import geopandas as gpd

pandas.set_option('display.width', -1)

ctxt = 'City'
crsinit = {'init': 'epsg:4326'}
newcrs_epsg = 2248

shapefile_dir = '/home/idies/workspace/Storage/raddick/Baltimore/shapefiles/'

thecmap = 'Purples' # color map
vmin, vmax = 0, 1  # color limits

neighborhood_data_source = 'openbaltimore'
num_neighborhoods_to_show = -1
show_neighborhood_names = False
show_streets = True

debug = 0

print('ok')

## Get neighborhood shapefiles from Tamas's database and Open Baltimore

In [ ]:
s = time.time()

print('Getting neighborhood shapefile data from Tamas\'s database...')
qnb = """
    select Neighborhood, geometry=Geog.ToString()
    from Neighborhoods 
"""
t_neighborhoods_df = CasJobs.executeQuery(qnb, ctxt)

t_neighborhoods_gdf = gpd.GeoDataFrame(t_neighborhoods_df, crs=crsinit, geometry=t_neighborhoods_df['geometry'].apply(wkt.loads))
t_neighborhoods_gdf = t_neighborhoods_gdf.rename(columns={'Neighborhood': 'neighborhood'})
t_neighborhoods_gdf = t_neighborhoods_gdf.set_index('neighborhood')
t_neighborhoods_gdf = t_neighborhoods_gdf.to_crs(epsg=newcrs_epsg)
e = time.time()

if (debug >= 1):
    print(t_neighborhoods_gdf.sample(2))
    print('\n')
print('Got {0:,.0f} neighborhoods in {1:.2f} seconds!'.format(len(t_neighborhoods_gdf), e-s))

print('Getting neighborhood shapefile data from Open Baltimore shapefiles...')

s = time.time()
nfile = shapefile_dir + 'neighborhoods_full/Neighborhoods.shp'
ob_neighborhoods_gdf = gpd.read_file(nfile, encoding='utf-8')
ob_neighborhoods_gdf = ob_neighborhoods_gdf.set_index('Name')
e = time.time()
ob_neighborhoods_gdf = ob_neighborhoods_gdf.to_crs(epsg=newcrs_epsg)

if (debug >= 1):
    print(ob_neighborhoods_gdf.sample(1).T)
    print('\n')
print('Got {0:,.0f} neighborhoods in {1:.2f} seconds!'.format(len(ob_neighborhoods_gdf), e-s))


if (show_streets):
    s = time.time()
    print('Reading street centerlines...')
    streets_filename = shapefile_dir + 'streets/streetcl.shp'
    streets_gdf = gpd.read_file(streets_filename)
    streets_gdf = streets_gdf.set_index('OBJECTID')
    streets_gdf = streets_gdf.to_crs(epsg=newcrs_epsg)
    e = time.time()
    print('Read {0:,.0f} street centerlines in {1:,.1f} seconds.'.format(len(streets_gdf), e-s))
print('Done!')

## Plot vacant buildings

In [ ]:
getBuildingsQuery = """
select r.ID, r.Block, r.Lot, r.StreetNameTrim, r.StreetTypeTrim, r.IsVacant, r.IsMCC, 
geometry=r.Geog.ToString(), b.MinNeighborhood, b.MaxNeighborhood
from RealGoodBuildings r
join Blocks b on r.Block = b.Block
where b.MinNeighborhood like '%s' or b.MaxNeighborhood like '%s'
"""

scalefactor = 1
fig, ax = plt.subplots(1, figsize=(48*scalefactor,48*scalefactor))
ax.set_aspect('equal')

if (show_streets):
    print('Plotting streets...')
    streets_gdf.plot(ax=ax, color='black', linewidth=1*scalefactor, alpha=0.25)
    
print('Plotting neighborhood boundaries...')
if (neighborhood_data_source == 'tamas'):
    labeling_neighborhoods_gdf = t_neighborhoods_gdf
    t_neighborhoods_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2*scalefactor)#, edgecolor='none', color='seashell')
elif (neighborhood_data_source == 'openbaltimore'):
    labeling_neighborhoods_gdf = ob_neighborhoods_gdf
    ob_neighborhoods_gdf.plot(ax=ax, color='none', edgecolor='black', linewidth=2*scalefactor)#, edgecolor='none', color='seashell')
else:
    print('ERROR: could not plot neighborhood boundaries. Please set a value for neighborhood_data_source.')
    
if (show_neighborhood_names):
    print('Plotting neighborhood labels...')
    for ix, thisrow in labeling_neighborhoods_gdf.iterrows():
        annotator = ix.replace('-', '- ')
        annotator = annotator.replace('/', '/ ')
        annotator = annotator.replace(' ', '\n')
        ax.annotate(annotator, 
                    xy=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                    xytext=(thisrow.geometry.centroid.x, thisrow.geometry.centroid.y), 
                    ha='center', va='center', fontsize=18*scalefactor, color='red')

print('Plotting vacant buildings...')
i = 1
if (num_neighborhoods_to_show == -1):
    num_neighborhoods_to_show = len(t_neighborhoods_gdf)
for thisnb, thisrow in t_neighborhoods_gdf.sample(num_neighborhoods_to_show).iterrows():
    if (debug == 0):
        if ((np.mod(i,5) == 0) | (i == num_neighborhoods_to_show)):
            print('Processing neighborhood {0:,.0f} of {1:,.0f}...'.format(i, num_neighborhoods_to_show))
    elif (debug == 1):
        print('Examining {0:}...'.format(thisnb.strip()))
    
    buildings_df = CasJobs.executeQuery(getBuildingsQuery % (thisnb.replace("'","''"),thisnb.replace("'","''")), ctxt)
    buildings_df = buildings_df.set_index('ID')

    buildings_gdf = gpd.GeoDataFrame(buildings_df, crs=crsinit, geometry=buildings_df['geometry'].apply(wkt.loads))
    buildings_gdf = buildings_gdf.to_crs(epsg=newcrs_epsg)
    if (debug == 1):
        print('Found {0:,.0f} vacant buildings from {1:,.0f} total.'.format(len(buildings_gdf[buildings_gdf['IsVacant'] == 1]), len(buildings_gdf)))
    #print(buildings_gdf.head(1).T)
    if (len(buildings_gdf) == 0):
        print('No buildings found for {0:}'.format(thisnb))
    buildings_gdf.plot(ax=ax, column='IsVacant', cmap=thecmap, vmin=vmin, vmax=vmax)
    i = i + 1

#ax.tick_params(axis='both')
ax.set_yticklabels([])
ax.set_xticklabels([])
plt.title('Vacant buildings in Baltimore (purple)', fontsize=64*scalefactor, y=1.01)

#ax.tick_params(axis='both', which='major', labelsize=48*scalescalefactor)
#ax.set_xlim((1410000,1420000))
#ax.set_ylim((585000,600000))

#plt.show()

plt.savefig('vacants.svg', format='svg')
print('Done!')


## Find real properties

In [ ]:
getPropertiesQuery = """
select top 50000 ID, Block, Lot, Ward, Section, Neighborhood,
ZoneCode, PermHome, BuildingNumber, Fraction, StreetName, StreetType,
UnitNumber, ZipCode, ZipExt, YearBuild, NoImprv,
OwnerAbbr, GroundRent, SDATLink, BlockPlat, geometry=Geog.ToString(),
GeogArea
from RealPropertiesAll
"""
properties_df = CasJobs.executeQuery(getPropertiesQuery, 'City')
properties_df = properties_df.set_index('ID')
print('done')

In [ ]:
q = 'select top 10 Block, Num, MinNeighborhood, MaxNeighborhood,\n'
q += 'MinLot, MaxLot, geography=Geog.ToString(), NumGeometries,\n'
q += 'Area, envelope_center=EnvelopeCenter.ToString(), EnvelopeAngle, EnvelopeAngleMeter,\n'
q += 'convexhull=chull.ToString()\n'
q += 'from Blocks'
df = CasJobs.executeQuery(q, 'City')
df

In [ ]:
q = 'select top 10 MinID, N, Nvac, Nmcc, Nvmc, Nown, IsSameStreet,\n'
q += 'geometry=Geog.ToString(), building_geography=BldgGeog.ToString()\n'
q += 'from SubBlockFaces'
df = CasJobs.executeQuery(q, 'City')
df

In [ ]:
q = """
select MinID, MinBlockLot=MIN(BlockLot), MaxBlockLot=MAX(BlockLot)
from RealGroups g
join RealPropertiesAll r on r.ID=g.ID
--where s.MinID=g.MinID and 
where r.Neighborhood like 'BLYTHEWOOD                      '
group by g.MinID
"""
#properties_df.groupby('Neighborhood').size()
properties_df.head(1).T

crsinit = {'init': 'epsg:4326'}
newcrs_epsg = 2248

properties_gdf = gpd.GeoDataFrame(properties_df, crs=crsinit, geometry=properties_df['geometry'].apply(wkt.loads))
properties_gdf = properties_gdf.to_crs(epsg=newcrs_epsg)

properties_gdf.sample(2).T

In [ ]:
#thequery = 'select id, block, lot, ward, section, neighborhood, zonecode, permhome,\n'
#thequery += 'fulladdress, fraction, unitnumber, zipcode, zipext, yearbuild, noimprv, ownerabbr, groundrent,\n'
#thequery += 'sdatlink, blockplat, shapestarea, shapestlength, shapetype, shapecoordinates\n'
#thequery += 'from RealPropertiesAll\n'

#df = CasJobs.executeQuery(thequery, 'City')
#df = df.set_index('id')
##print(thequery)
#df

# Get neighborhood names and shapefiles

In [ ]:
#  query template for neighborhood shape
debug = 1

qnb = """
    select Neighborhood, geometry=Geog.ToString()
    from Neighborhoods 
"""

# query template for number of vacants, etc., in subblockfaces
qry = """
    select s.MinID, N, Nvac, Nmcc, Nvmc, Nown, Fvmc=convert(float,Nvmc)/N
    , MinBlockLot, MaxBlockLot
    , IsSameStreet=coalesce(IsSameStreet,1)
    , geometry=BldgGeog.ToString()
    from SubBlockFaces s
        cross apply (
            select MinID, MinBlockLot=MIN(BlockLot), MaxBlockLot=MAX(BlockLot)
            from RealGroups g
                join RealPropertiesAll r on r.ID=g.ID
            where s.MinID=g.MinID and Neighborhood like '%s'
            group by g.MinID
        ) x
    where Nvac > 0
"""

ctxt = 'City'

crsinit = {'init': 'epsg:4326'}
newcrs_epsg = 2248

cmap = 'cubehelix_r' # color map
vmin, vmax = 0, 1  # color limits

# helper function to get geo data
def get_vacants_in_neighborhood(qry, geocolumn='geometry', ctxt=ctxt, crs=newcrs_epsg):
    df = CasJobs.executeQuery(qry, ctxt)
    crsinit = {'init': 'epsg:4326'}
    gdf = gpd.GeoDataFrame(df, crs=crsinit, geometry=df[geocolumn].apply(wkt.loads))
    return gdf.to_crs(epsg=crs)

print('ok')

In [ ]:
neighborhoods_df = CasJobs.executeQuery(qnb, ctxt)

neighborhoods_gdf = gpd.GeoDataFrame(neighborhoods_df, crs=crsinit, geometry=neighborhoods_df['geometry'].apply(wkt.loads))
neighborhoods_gdf = neighborhoods_gdf.rename(columns={'Neighborhood': 'neighborhood'})
neighborhoods_gdf = neighborhoods_gdf.set_index('neighborhood')
neighborhoods_gdf = neighborhoods_gdf.to_crs(epsg=newcrs_epsg)
if (debug >= 1):
    print(neighborhoods_gdf.sample(2))
else:
    print('ok')

In [ ]:
for thisnb, thisrow in neighborhoods_gdf.iterrows():
    
    vacants_df = get_vacants_in_neighborhood(qry % thisnb.replace("'", "''"))
    vacants_df['Nvmc'] = pandas.to_numeric(vacants_df['Nvmc'], errors='coerce')
    vacants_df['Fvmc'] = pandas.to_numeric(vacants_df['Fvmc'], errors='coerce')
    vacants_df[['Nvmc', 'N']] = vacants_df[['Nvmc', 'N']].fillna(0)
    
    if (vacants_df['N'].sum() > 0):
        if (debug >= 2):
            print('Loading {0:}...'.format(thisnb.replace("'", "''").strip()))
            print('Found {0:,.0f} vacant units ({1:.1%} of all units).'.format(
                vacants_df['Nvmc'].sum(), 
                (vacants_df['Nvmc'].sum() / vacants_df['N'].sum())
            ))
    else:
        if (debug >= 1):
            print('No units found in {0:}!'.format(thisnb))
    #print('\n')
    #vacants_df = get_vacants_in_neighborhood(qry % thisnb.replace("'", "''"))

    
#vacants_df.index.name = 'rownum'
    #print ('Loading ' , i , '/', df.Neighborhood.size, ' : ', n)
print('Done!')

In [ ]:
fig, ax = plt.subplots(1, figsize=(16,12))
ax.set_aspect('equal')

neighborhoods_gdf.plot(ax=ax, linewidth=0.05, edgecolor='none', color='seashell')
vacants_gdf.plot(ax=ax, linewidth=0.05, edgecolor='black', column='Fvmc', cmap=cmap, vmin=vmin, vmax=vmax)
plt.show()

In [ ]:
#print(qry % 'BLYTHEWOOD                      ')
#q = 'select top 20 ID, BlockLot, Block, Lot, Ward, Section, Neighborhood, '
#q += 'ZoneCode, PermHome, FullAddress, StreetName, StreetType, BuildingNumber, '
#q += 'Fraction, UnitNumber, ZipCode, ZipExt, YearBuild, NoImprv, '
#q += 'OwnerAbbr, GroundRent, SDATLink, BlockPlat, ShapeSTArea, ShapeSTLength, '
#q += 'ShapeType , ShapeCoordinates, GeogArea '#, GeogCenter '





#rpdf.sample(2).T

#qb = 'select Block, Num, MinNeighborhood, MaxNeighborhood, '
#qb += 'MinLot, MaxLot, geography = Geog.ToString(), NumGeometries '
#qb += 'Area, ec = EnvelopeCenter.ToString(), EnvelopeAngle, EnvelopeAngleMeter, thechull = CHull.ToString() '
qb = 'select Block, count(*) '
qb += 'from Blocks '
qb += 'group by Block'
qbdf = CasJobs.executeQuery(qb, 'City')
#qbdf.sample(2).T
qbdf

qs = 'select top 20 MinID, N, Nvac, Nmcc, Nvmc, Nown, IsSameStreet, geography=Geog.ToString(), '
qs += 'building_geography=BldgGeog.ToString() '
qs += 'from SubBlockFaces'
sdf = CasJobs.executeQuery(qs, 'City')
sdf


qv = 'select ID, BlockLot, FullAddress, Notice, DtIns, DtNote, DtExp, VacInd, '
qv += 'Rental, LienAmt, GlobalID, DateIns, DateNote, DateExp, '
qv += 'geopoint = GeogPoint.ToString() '
qv += 'from VacantBuildings'
vdf = CasJobs.executeQuery(qv, 'City')
vdf = vdf.set_index('ID')
vdf.sample(3).T




buildings_df = CasJobs.executeQuery(qbuildings, 'City')
buildings_df = buildings_df.set_index('ID')




properties_df = CasJobs.executeQuery(qproperties, 'City')
properties_df = properties_df.set_index('ID')

#buildings_df.sample(3).T
properties_df.sample(1).T
#properties_df['BlockPlat'].sample(1).T.tolist()

#df = df.set_index('ID')
#sdf.head(3).T
#MinID, MinBlockLot=MIN(BlockLot), MaxBlockLot=MAX(BlockLot)
#            from RealGroups g
#                join RealPropertiesAll r on r.ID=g.ID
#            where s.MinID=g.MinID and Neighborhood like 'BLYTHEWOOD                      '
#            group by g.MinID

In [ ]:

vacant_query = """
    select s.MinID, N, Nvac, Nmcc, Nvmc, Nown, Fvmc=convert(float,Nvmc)/N
    , MinBlockLot, MaxBlockLot
    , IsSameStreet=coalesce(IsSameStreet,1)
    , geometry=BldgGeog.ToString()
    from SubBlockFaces s
        cross apply (
            select MinID, MinBlockLot=MIN(BlockLot), MaxBlockLot=MAX(BlockLot)
            from RealGroups g
                join RealPropertiesAll r on r.ID=g.ID
            where s.MinID=g.MinID and Neighborhood like '%s'
            group by g.MinID
        ) x
    where Nvac > 0
"""
ctxt = 'City'
crsinit = {'init': 'epsg:4326'}
crs=2248

In [ ]:
df = CasJobs.executeQuery(vacant_query, ctxt)
gdf = gpd.GeoDataFrame(df, crs=crsinit, geometry=df['geometry'].apply(wkt.loads))
gdf = gdf.to_crs(epsg=crs)
gdf.head(2)

In [ ]:

#print(qry)
cnt = 1
for thisnbr, thisrow in neighborhoods_gdf.iterrows():
    print('Processing neighborhood {0:,.0f} of {1:,.0f}: {2:}...'.format(cnt, neighborhoods_gdf.size, thisnbr.strip()))
    #print((vacant_query % thisnbr).replace("'", "''"))
    thisgdf = GeoQuery(vacant_query.format(thisnbr))#.replace("'", "''"))
    print(thisgdf['Fvmc'])
    #print(df.sample(1))
    cnt = cnt + 1
    #print('\n')
#df.sample(3)
thisgdf

#for i,n in enumerate(df.Neighborhood[:]).sample(3):
    #print ('Loading ' , i , '/', df.Neighborhood.size, ' : ', n)
    #sbf = GeoQuery(qry % n.replace("'", "''"))
    #print ('>>>', sbf.shape[0:2])
    #sbf.plot(ax=ax, linewidth=0.05, edgecolor='black', column='Fvmc', cmap=cmap, vmin=vmin, vmax=vmax)


In [ ]:
cmap = 'cubehelix_r' # color map
vmin, vmax = 0, 1  # color limits

fig, ax = plt.subplots(1, figsize=(16,12))
ax.set_aspect('equal')
neighborhoods_gdf.plot(ax=ax, linewidth=0.05, color='seashell')
gdf.plot(ax=ax, column='Fvmc')  
plt.show()

In [ ]:
qproperties = 'select ID, Block, Lot, Ward, Section, Neighborhood,\n'
qproperties += 'ZoneCode, PermHome, BuildingNumber, Fraction, StreetName, StreetType,\n'
qproperties += 'UnitNumber, ZipCode, ZipExt, YearBuild, NoImprv,\n'
qproperties += 'OwnerAbbr, GroundRent, SDATLink, BlockPlat,\n'
qproperties += 'GeogArea\n'
#qproperties += 'geography=Geog.ToString(), GeogArea, center=GeogCenter.ToString()\n'
qproperties += 'into MyDB..RealProperties\n'
qproperties += 'from RealPropertiesAll'

jobid = CasJobs.submitJob(qproperties, 'City')
print(jobid)

In [ ]:
cmap = 'cubehelix_r' # color map
vmin, vmax = 0, 1  # color limits

fig, ax = plt.subplots(1, figsize=(16,12))
ax.set_aspect('equal')

# plot neighborhood outlines (union of parcels)
nbr = GeoQuery(qnb)
nbr.plot(ax=ax, linewidth=0.05, edgecolor='none', color='seashell')

# plot contiguous houses with vacancy
for i,n in enumerate(df.Neighborhood[:]).sample(3):
    print ('Loading ' , i , '/', df.Neighborhood.size, ' : ', n)
    sbf = GeoQuery(qry % n.replace("'", "''"))
    print ('>>>', sbf.shape[0:2])
    sbf.plot(ax=ax, linewidth=0.05, edgecolor='black', column='Fvmc', cmap=cmap, vmin=vmin, vmax=vmax)
    
#ax.axis('off')
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)  
plt.tight_layout()
#plt.savefig('_city.pdf')
#plt.savefig('city.svg', format='svg')
#print('Done')
plt.show()